In [13]:
pip install numpy --upgrade

     --------------------------------------- 14.9/14.9 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [15]:
from pprint import pprint
import functools

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
from sklearn.metrics import confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm.notebook import tqdm

import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np
import csv


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
def labeliser_tweet(df, nb_tweets=None, random_sample=False):

    # Charger le fichier "label.csv" pour vérifier les tweets déjà labelisés
    tweets_labelises = set()
    try:
        with open('label.csv', 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                tweets_labelises.add(row['tweet_id'])
    except FileNotFoundError:
        pass

    # Sélectionner les tweets à labeliser
    if random_sample:
        if nb_tweets:
            tweets_a_labeliser = df.drop(tweets_labelises).sample(n=nb_tweets)
        else:
            tweets_a_labeliser = df.drop(tweets_labelises)
    else:
        if nb_tweets:
            tweets_a_labeliser = df.drop(tweets_labelises).head(nb_tweets)
        else:
            tweets_a_labeliser = df.drop(tweets_labelises)

    # Ouvrir le fichier "label.csv" en mode ajout, pour ne pas écraser les données précédentes
    with open('label.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Si le fichier est vide, ajouter une ligne d'en-tête
        if csvfile.tell() == 0:
            writer.writerow(['tweet_id', 'text', 'label'])

        # Boucle pour labeliser chaque tweet sélectionné
        for tweet_id, row in tweets_a_labeliser.iterrows():
            text = row['text']
            labelisation = input(f'Label pour le tweet "{text}": ')

            # Écrire la labelisation dans le fichier "label.csv"
            writer.writerow([tweet_id, text, labelisation])


In [ ]:
import pandas as pd

def labeliser_tweet(df, nb_tweets, label_file):
    # Charger les tweets déjà labelisés
    try:
        with open(label_file, mode='r') as f:
            reader = csv.DictReader(f)
            tweets_labelises = set(row['tweet_id'] for row in reader)
    except FileNotFoundError:
        tweets_labelises = set()

    # Sélectionner les tweets à labeliser
    df_a_labeliser = df[~df.index.isin(tweets_labelises)].sample(n=nb_tweets, random_state=42)

    # Itérer sur les tweets sélectionnés et demander le label à l'utilisateur
    for tweet_id, row in df_a_labeliser.iterrows():
        print(row['text'])
        labelisation = input(f'Label pour {text}: ')
        df.loc[tweet_id, 'label'] = labelisation

    # Sauvegarder les labels dans le fichier CSV
    df.to_csv(label_file, index_label='tweet_id')

    print(f"{nb_tweets} tweets ont été labelisés et ajoutés au fichier {label_file}")


In [3]:
def labeliser_tweet(df, nb_tweets=5):
    
    # Vérifier si le fichier "label.csv" existe et charger les tweet_id déjà labelisés
    tweets_labelises = set()
    try:
        with open('label.csv', mode='r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                tweets_labelises.add(row['tweet_id'])
    except FileNotFoundError:
        pass
    
    # Sélectionner un échantillon aléatoire de tweets non encore labelisés
    df_a_labeliser = df[~df.index.isin(tweets_labelises)].sample(n=nb_tweets, random_state=42)
    
    # Labeliser les tweets sélectionnés et enregistrer les labels dans un fichier CSV
    with open('label.csv', mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(['tweet_id', 'text', 'label'])
        for tweet_id, text in df_a_labeliser['text'].items():
            label = input(f'Label pour le tweet suivant :\n{text}\n')
            # Écrire les données labelisées dans le fichier CSV
            writer.writerow([tweet_id, text, label])


In [4]:
df = pd.read_csv('data/data_concat_clean/base_annotation_fev23.csv', index_col='tweet_id').drop('Unnamed: 0', axis=1)

In [5]:
labeliser_tweet(df, nb_tweets=5)

Label pour le tweet suivant :
Vous ne connaissez pas de collègues qui ont été violées en toute impunité par un supérieur hiérarchique, donc ?  Mince. Mais vous pouvez peut-être vous informer, non ? Ce n'est pas la première fois que je vous le demande sur Twitter. 🙉🙈🙊  #metoo  #moiaussiEducationNationale https://t.co/R1IqhWgQKl
0
Label pour le tweet suivant :
-Jsais profiter de ma notoriété pour agresser sexuellement qui jveux puis prétendre à l'absence d'humour chez mes victimes pour légitimer mes actes!  -Bienvenue chez Canal+ !!!!!  #balancetonporc   #PierreMenesOut https://t.co/0YyApNM3jh
0
Label pour le tweet suivant :
#metoo #metooinceste  Excusez-moi, j'avais prévu un texte si je craquerais.
0
Label pour le tweet suivant :
#MeToo Je zappe : les femmes défendues à la télévision par un imbécile presqu'assexué, c'est trop fort... Hey, les godiches, vous vous prenez pour qui ? QUI ?
0
Label pour le tweet suivant :
#metooinceste #NousToutes @memoiretrauma  Je n'avais pas d'âge. Mon pè

In [6]:
df_label = pd.read_csv('label.csv')

In [7]:
df_label

,tweet_id,text,label
0,1282587829357490179,Vous ne connaissez pas de collègues qui ont ét...,0
1,1374666678533951488,-Jsais profiter de ma notoriété pour agresser ...,0
2,1463447795877265410,"#metoo #metooinceste Excusez-moi, j'avais pré...",0
3,1305966401887371264,#MeToo Je zappe : les femmes défendues à la té...,0
4,1350452454782070786,#metooinceste #NousToutes @memoiretrauma Je n...,0


In [8]:
# Defining constants
epochs = 5
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

NameError: name 'CamembertTokenizer' is not defined

In [ ]:
text = df['text'].to_list()
labels = df['label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

KeyError: ignored

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

NameError: ignored

In [ ]:
for _ in trange(epochs, desc="Epoch"):  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels.long())
        loss = outputs[0]
        train_loss_set.append(loss.item())    
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs =  model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels.long())
            loss, logits = outputs[:2]
    
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]


NameError: ignored